In [1]:
import os

os.chdir('../../')

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datasetsforecast.m3 import M3

from models.auto.AutoTimeMoe import AutoTimeMoe
from neuralforecast.tsdataset import TimeSeriesDataset

c:\Users\ricar\mixture_of_experts_time_series\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-05 22:12:08,697	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-03-05 22:12:08,900	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [ ]:
# load m3
df = M3.load(
            directory="C:\\Users\\ricar\\mixture_of_experts_time_series\\data\\m3",
            group="Monthly")[0]

def train_test_split(df: pd.DataFrame, horizon: int):
    """Split the dataframe into training and test sets by horizon."""
    groups = df.groupby('unique_id')
    train_list, test_list = [], []
    for _, group_df in groups:
        group_df = group_df.sort_values('ds')
        train_list.append(group_df.head(-horizon))
        test_list.append(group_df.tail(horizon))
    train_df = pd.concat(train_list).reset_index(drop=True)
    test_df = pd.concat(test_list).reset_index(drop=True)
    return train_df, test_df

# df['ds'] = pd.to_datetime(df['ds']).astype(int)

y_train_df, y_test_df = train_test_split(df, horizon=18)

dataset, *_ = TimeSeriesDataset.from_df(y_train_df)

c:\Users\ricar\mixture_of_experts_time_series\venv\Lib\site-packages\datasetsforecast\m3.py:108: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  freq = pd.tseries.frequencies.to_offset(class_group.freq)


In [4]:
model = AutoTimeMoe(h=18, num_samples=20, backend="optuna")

model.fit(dataset)

[I 2025-03-05 22:10:31,413] A new study created in memory with name: no-name-1cd3a9ee-8876-4c71-a236-beeae2f8c41a
c:\Users\ricar\mixture_of_experts_time_series\venv\Lib\site-packages\neuralforecast\common\_base_auto.py:291: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  v = trial.suggest_loguniform(k, v.lower, v.upper)
INFO:lightning_fabric.utilities.seed:Seed set to 12
[W 2025-03-05 22:10:31,424] Trial 0 failed with parameters: {'hidden_size': 64, 'n_head': 8, 'learning_rate': 0.0854067553927715, 'scaler_type': None, 'batch_size': 256, 'windows_batch_size': 256, 'intermediate_size': 1000, 'random_seed': 12, 'num_experts_per_tok': 2, 'num_experts': 4, 'input_size': 72, 'step_size': 18} because of the following error: TypeError("unsupported operand type(s) for //: 'int' and 'NoneType'").
Traceback (most recent call last):
  File 

TypeError: unsupported operand type(s) for //: 'int' and 'NoneType'

In [ ]:
# get best model config
test_dataset, *_ = TimeSeriesDataset.from_df(y_test_df)

y_hat = model.predict(dataset=test_dataset)

y_hat